# Big Red Soundscape
Notebook Playground 2  
_by Ahmed Sultan and Carly Hu, for Cornell Data Journal_

In [11]:
import json
import math

import numpy as np
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import matplotlib.pyplot as plt
import seaborn as sns

In [12]:
# update access keys here for Spotify API here :) 
client_id = 'f7661e34c0924317a8b928c0f977e6a7'
client_secret = 'd068ec3f0a98490e9c32be820916e934'

# sp is how we interface with the Spotify API
# read the docs here: https://spotipy.readthedocs.io/en/2.22.1/
sp = spotipy.Spotify(client_credentials_manager=\
                     SpotifyClientCredentials(client_id, client_secret))

<h1> Let's load in our survey data! </h1>

In [13]:
students_df = pd.read_csv('datacollection_050323.csv')
students_df.head()

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,Q8_3,Q8_4,Q9,Q10,Q11_1,Q11_2,Q11_3,Q11_4,Q11_5,Q12
0,Start Date,End Date,Response Type,IP Address,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,Recipient Last Name,...,How do you perceive yourself? - Introverted (0...,How do you perceive yourself? - Logical (0) vs...,Link your Spotify profile here—this provides u...,Link a Spotify playlist that captures your mus...,If you have Spotify playlists for the followin...,If you have Spotify playlists for the followin...,If you have Spotify playlists for the followin...,If you have Spotify playlists for the followin...,If you have Spotify playlists for the followin...,If you would like the chance to win a $15 gift...
1,"{""ImportId"":""startDate"",""timeZone"":""America/De...","{""ImportId"":""endDate"",""timeZone"":""America/Denv...","{""ImportId"":""status""}","{""ImportId"":""ipAddress""}","{""ImportId"":""progress""}","{""ImportId"":""duration""}","{""ImportId"":""finished""}","{""ImportId"":""recordedDate"",""timeZone"":""America...","{""ImportId"":""_recordId""}","{""ImportId"":""recipientLastName""}",...,"{""ImportId"":""QID18_3""}","{""ImportId"":""QID18_4""}","{""ImportId"":""QID19_TEXT""}","{""ImportId"":""QID20_TEXT""}","{""ImportId"":""QID21_1""}","{""ImportId"":""QID21_2""}","{""ImportId"":""QID21_3""}","{""ImportId"":""QID21_4""}","{""ImportId"":""QID21_5""}","{""ImportId"":""QID24_TEXT""}"
2,2023-04-12 15:22:47,2023-04-12 15:24:16,IP Address,*******,100,89,True,2023-04-12 15:24:17,R_2YtbSvySpz2s87b,*******,...,4,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-04-12 15:22:48,2023-04-12 15:26:13,IP Address,*******,100,205,True,2023-04-12 15:26:14,R_24ixQUbGn1AHnjs,*******,...,7,4,https://open.spotify.com/user/6cjzkudbvmwfe963...,https://open.spotify.com/playlist/5Kwx1SSB0pol...,NaN,https://open.spotify.com/playlist/37HUliOloWGE...,NaN,https://open.spotify.com/playlist/37HUliOloWGE...,https://open.spotify.com/playlist/5Kwx1SSB0pol...,NaN
4,2023-04-12 15:22:51,2023-04-12 15:26:40,IP Address,*******,100,229,True,2023-04-12 15:26:41,R_aWSgue6VU9HyHMR,*******,...,8,5,https://open.spotify.com/user/vqlx3sn88fh9295c...,https://open.spotify.com/playlist/6huxxOvKkbRu...,NaN,NaN,NaN,NaN,NaN,NaN


<h1> Data Cleaning </h1>

In [14]:
# get rid of some silly columns
students_df = students_df.drop(columns=['StartDate', 'EndDate',
                                        'Status','IPAddress','Duration (in seconds)','RecordedDate',
                                        'ResponseId','RecipientLastName','RecipientFirstName',
                                        'RecipientEmail', 'ExternalReference','LocationLatitude', 
                                        'LocationLongitude','DistributionChannel', 'UserLanguage'])

students_df # so many NaNs oh god

,Progress,Finished,Q1,Q2,Q3,Q4,Q7,Q5,Q6,Q6.1,...,Q8_3,Q8_4,Q9,Q10,Q11_1,Q11_2,Q11_3,Q11_4,Q11_5,Q12
0,Progress,Finished,What college are you primarily affiliated with?,What is your (primary) major?,What year are you?,Gender,Extracurricular involvements (select all that ...,Are you an international or domestic (U.S.) st...,What country are you from?,What state are you from?,...,How do you perceive yourself? - Introverted (0...,How do you perceive yourself? - Logical (0) vs...,Link your Spotify profile here—this provides u...,Link a Spotify playlist that captures your mus...,If you have Spotify playlists for the followin...,If you have Spotify playlists for the followin...,If you have Spotify playlists for the followin...,If you have Spotify playlists for the followin...,If you have Spotify playlists for the followin...,If you would like the chance to win a $15 gift...
1,"{""ImportId"":""progress""}","{""ImportId"":""finished""}","{""ImportId"":""QID5""}","{""ImportId"":""QID14""}","{""ImportId"":""QID8""}","{""ImportId"":""QID10""}","{""ImportId"":""QID17""}","{""ImportId"":""QID11""}","{""ImportId"":""QID15""}","{""ImportId"":""QID16""}",...,"{""ImportId"":""QID18_3""}","{""ImportId"":""QID18_4""}","{""ImportId"":""QID19_TEXT""}","{""ImportId"":""QID20_TEXT""}","{""ImportId"":""QID21_1""}","{""ImportId"":""QID21_2""}","{""ImportId"":""QID21_3""}","{""ImportId"":""QID21_4""}","{""ImportId"":""QID21_5""}","{""ImportId"":""QID24_TEXT""}"
2,100,True,(CALS) College of Agriculture and Life Sciences,Biometry and Statistics,"Undergraduate, 4th year (2023)",Male,"Project team,Other club/team",Domestic,NaN,Florida,...,4,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100,True,(AS) College of Arts and Sciences,Computer Science,"Undergraduate, 1st year (2026)",Male,Other club/team,International,Korea South,NaN,...,7,4,https://open.spotify.com/user/6cjzkudbvmwfe963...,https://open.spotify.com/playlist/5Kwx1SSB0pol...,NaN,https://open.spotify.com/playlist/37HUliOloWGE...,NaN,https://open.spotify.com/playlist/37HUliOloWGE...,https://open.spotify.com/playlist/5Kwx1SSB0pol...,NaN
4,100,True,(AS) College of Arts and Sciences,Government,"Undergraduate, 1st year (2026)",Female,Professional club,International,Indonesia,NaN,...,8,5,https://open.spotify.com/user/vqlx3sn88fh9295c...,https://open.spotify.com/playlist/6huxxOvKkbRu...,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,100,True,(CALS) College of Agriculture and Life Sciences,Biometry and Statistics,"Undergraduate, 2nd year (2025)",Female,"Professional club,Project team,Other club/team",Domestic,NaN,New York,...,4,5,https://open.spotify.com/user/kjtai?si=76189a3...,https://open.spotify.com/playlist/74jVE9KAr6Zc...,NaN,https://open.spotify.com/playlist/5yZMQxX51f4S...,NaN,NaN,https://open.spotify.com/playlist/0SkPSwqZVAhS...,NaN
152,100,True,(AS) College of Arts and Sciences,Computer Science,"Undergraduate, 1st year (2026)",Male,Other club/team,Domestic,NaN,Maryland,...,3,3,https://open.spotify.com/user/97mi316dq36dfama...,https://open.spotify.com/playlist/66wgLF7XDGqk...,NaN,NaN,NaN,NaN,NaN,NaN
153,100,True,(ENG) College of Engineering,Biomedical Engineering,"Undergraduate, 1st year (2026)",Female,Other club/team,Domestic,NaN,New York,...,4,5,https://open.spotify.com/user/jec5zqzb11o9e0el...,https://open.spotify.com/playlist/37i9dQZF1Epg...,NaN,NaN,NaN,NaN,NaN,ac2566@cornell.edu
154,7,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# drop anyone who didn't link a profile properly

students_df = students_df.dropna(0, subset=['Q9'])
students_df = students_df[students_df["Q9"].str.contains('https://open.spotify.com/user')]

# print our questions, then drop row of questions
students_df = students_df.drop(0)

# reset index 
students_df = students_df.reset_index()

# ok what do we have now
print('Number of linked spotify profiles: ' + str(students_df.shape[0]))
students_df

Number of linked spotify profiles: 50


/var/folders/qg/5ts3y3kd41l7546h43d5mdy00000gn/T/ipykernel_4622/4291974516.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  students_df = students_df.dropna(0, subset=['Q9'])


,index,Progress,Finished,Q1,Q2,Q3,Q4,Q7,Q5,Q6,...,Q8_3,Q8_4,Q9,Q10,Q11_1,Q11_2,Q11_3,Q11_4,Q11_5,Q12
0,3,100,True,(AS) College of Arts and Sciences,Computer Science,"Undergraduate, 1st year (2026)",Male,Other club/team,International,Korea South,...,7,4,https://open.spotify.com/user/6cjzkudbvmwfe963...,https://open.spotify.com/playlist/5Kwx1SSB0pol...,NaN,https://open.spotify.com/playlist/37HUliOloWGE...,NaN,https://open.spotify.com/playlist/37HUliOloWGE...,https://open.spotify.com/playlist/5Kwx1SSB0pol...,NaN
1,4,100,True,(AS) College of Arts and Sciences,Government,"Undergraduate, 1st year (2026)",Female,Professional club,International,Indonesia,...,8,5,https://open.spotify.com/user/vqlx3sn88fh9295c...,https://open.spotify.com/playlist/6huxxOvKkbRu...,NaN,NaN,NaN,NaN,NaN,NaN
2,7,100,True,(ENG) College of Engineering,Cognitive Science,"Undergraduate, 1st year (2026)",Male,"Project team,Other club/team",Domestic,NaN,...,4,4,https://open.spotify.com/user/m3l6ktgr5im6yorm...,https://open.spotify.com/playlist/3Tp8L5xWN4OQ...,https://open.spotify.com/playlist/4ueOZx7jYJC1...,NaN,NaN,NaN,NaN,NaN
3,8,100,True,(ILR) School of Industrial and Labor Relations,Industrial and Labor Relations,"Undergraduate, 3rd year (2024)",Female,"Professional club,Other club/team",Domestic,NaN,...,5,7,https://open.spotify.com/user/t4llwr7d08bca91r...,https://open.spotify.com/playlist/76R0Tg5FCTrb...,NaN,NaN,NaN,NaN,NaN,NaN
4,9,100,True,(ENG) College of Engineering,Computer Science,"Undergraduate, 4th year (2023)",Male,Project team,Domestic,NaN,...,5,3,https://open.spotify.com/user/ejlity?si=B7MbIe...,https://open.spotify.com/playlist/07epaq8mPbCB...,NaN,NaN,NaN,NaN,NaN,NaN
5,13,100,True,(AS) College of Arts and Sciences,Mathematics,"Undergraduate, 2nd year (2025)",Female,"Professional club,Other club/team",Domestic,NaN,...,1,5,https://open.spotify.com/user/aaa9rnpre89bf2i0...,https://open.spotify.com/playlist/4WjCgwiirz98...,NaN,https://open.spotify.com/playlist/75P8aVPlabMc...,NaN,NaN,NaN,NaN
6,15,100,True,(AS) College of Arts and Sciences,English,"Undergraduate, other",Female,"Professional club,Other club/team",Domestic,NaN,...,6,7,https://open.spotify.com/user/deadpan.typicaln...,https://open.spotify.com/playlist/6O8Gd241tsCl...,https://open.spotify.com/playlist/3cgTaDVasXgE...,https://open.spotify.com/playlist/3tGQUB0EjqFy...,NaN,https://open.spotify.com/playlist/6HfiVdghosyM...,https://open.spotify.com/playlist/3nUAsSk8rEWK...,NaN
7,16,100,True,(CALS) College of Agriculture and Life Sciences,Information Science,"Undergraduate, 4th year (2023)",Female,Other club/team,Domestic,NaN,...,5,2,https://open.spotify.com/user/fluttershyfatima...,https://open.spotify.com/playlist/48ufj7347Mnw...,NaN,NaN,NaN,NaN,NaN,NaN
8,17,100,True,(AS) College of Arts and Sciences,Government,"Undergraduate, 3rd year (2024)",Female,Other club/team,Domestic,NaN,...,4,8,https://open.spotify.com/user/1283949946?si=nj...,https://open.spotify.com/playlist/3FLTO5QNCWTg...,https://open.spotify.com/playlist/6M9RziV7gkFz...,https://open.spotify.com/playlist/32hpqnOCX6YU...,https://open.spotify.com/playlist/4BIbSL8yR154...,https://open.spotify.com/playlist/2x2kvO5kP1e3...,https://open.spotify.com/playlist/0PwT4umg8ZSa...,NaN
9,18,100,True,(ENG) College of Engineering,Operations Research and Engineering,"Undergraduate, 3rd year (2024)",Male,"Professional club,Other club/team",Domestic,NaN,...,4,3,https://open.spotify.com/user/31m3onadeh4wxucg...,https://open.spotify.com/playlist/26zMq20Djd7R...,https://open.spotify.com/playlist/7F0HcjeuLDVL...,NaN,NaN,NaN,NaN,NaN


<h1> Getting spotify usernames... </h1>

In [6]:
# let's get a list of spotify usernames

usernames = students_df['Q9']

stripped_usernames = []

for i in usernames:
    i=i.replace("https://open.spotify.com/user/", "")
    i=i.split('?si',1)[0]
    stripped_usernames.append(i)

    i =+ 1

print(len(stripped_usernames))
stripped_usernames

50


['6cjzkudbvmwfe963clvemzwkj',
 'vqlx3sn88fh9295cpn4z7toip',
 'm3l6ktgr5im6yormbf36etw78',
 't4llwr7d08bca91rqn1ro2aau',
 'ejlity',
 'aaa9rnpre89bf2i0tx7ihyem0',
 'deadpan.typicalnerdbird1492',
 'fluttershyfatima',
 '1283949946',
 '31m3onadeh4wxucgckv3fhcyeoqu',
 'sir.frances.drake',
 'seemslikehope',
 'tiffxny.lee',
 'sharktiger',
 'bn8wfbeyvujlinqcmoj41nymr',
 'owllson',
 'alexabarton16',
 'mtaakmfvi2rkrcrkbg3qsv1b0',
 '31tczmefxwht47ndcx5rnqqztaaa',
 'alayna.earl',
 'n8hn1814',
 'd4iyjnkubkj5j1rmdr6rxdp1y',
 'bad_mug',
 'sup.arna',
 'kobitravels',
 'jessica20022',
 'o035tufj1j2to59df208ozh5k',
 'meepsymo',
 'vbvb101',
 'dihbuj006e36pz2kkpeiyr9it',
 'clarebelman',
 '1re58jm2ppu8vkex6p2lqkyvt',
 'andreacmauri',
 'olivia-1212',
 '12124545665',
 '12139305999',
 'jhkmaggie',
 'eirian_h',
 'lenatp808',
 'ayamtume',
 '313sqjo7imdsqz2sub73bwrnsdlm',
 'whc30ixinkb9qw5xkfcn6pg5z',
 'seanlamacchia',
 '3qu9523ujief7vb0wheia8yxm',
 'kaybae.',
 '317fva6tkhzg3zhuz7jqbticmoqq',
 'zzigak',
 'kjtai',


In [7]:
students_df['username'] = stripped_usernames # adding username column

students_df.head() # READY FOR ANALYSIS WOOOO



,index,Progress,Finished,Q1,Q2,Q3,Q4,Q7,Q5,Q6,...,Q8_4,Q9,Q10,Q11_1,Q11_2,Q11_3,Q11_4,Q11_5,Q12,username
0,3,100,True,(AS) College of Arts and Sciences,Computer Science,"Undergraduate, 1st year (2026)",Male,Other club/team,International,Korea South,...,4,https://open.spotify.com/user/6cjzkudbvmwfe963...,https://open.spotify.com/playlist/5Kwx1SSB0pol...,NaN,https://open.spotify.com/playlist/37HUliOloWGE...,NaN,https://open.spotify.com/playlist/37HUliOloWGE...,https://open.spotify.com/playlist/5Kwx1SSB0pol...,NaN,6cjzkudbvmwfe963clvemzwkj
1,4,100,True,(AS) College of Arts and Sciences,Government,"Undergraduate, 1st year (2026)",Female,Professional club,International,Indonesia,...,5,https://open.spotify.com/user/vqlx3sn88fh9295c...,https://open.spotify.com/playlist/6huxxOvKkbRu...,NaN,NaN,NaN,NaN,NaN,NaN,vqlx3sn88fh9295cpn4z7toip
2,7,100,True,(ENG) College of Engineering,Cognitive Science,"Undergraduate, 1st year (2026)",Male,"Project team,Other club/team",Domestic,NaN,...,4,https://open.spotify.com/user/m3l6ktgr5im6yorm...,https://open.spotify.com/playlist/3Tp8L5xWN4OQ...,https://open.spotify.com/playlist/4ueOZx7jYJC1...,NaN,NaN,NaN,NaN,NaN,m3l6ktgr5im6yormbf36etw78
3,8,100,True,(ILR) School of Industrial and Labor Relations,Industrial and Labor Relations,"Undergraduate, 3rd year (2024)",Female,"Professional club,Other club/team",Domestic,NaN,...,7,https://open.spotify.com/user/t4llwr7d08bca91r...,https://open.spotify.com/playlist/76R0Tg5FCTrb...,NaN,NaN,NaN,NaN,NaN,NaN,t4llwr7d08bca91rqn1ro2aau
4,9,100,True,(ENG) College of Engineering,Computer Science,"Undergraduate, 4th year (2023)",Male,Project team,Domestic,NaN,...,3,https://open.spotify.com/user/ejlity?si=B7MbIe...,https://open.spotify.com/playlist/07epaq8mPbCB...,NaN,NaN,NaN,NaN,NaN,NaN,ejlity


In [9]:
students_df.to_csv('all_cornell_demographics_050323.csv', index=False)

# Helper functions for spotify processing

In [34]:
def user_playlist_tracks_full(user, playlist_id=None, fields=None):
    """ Get full details of the tracks of a playlist owned by a user.
        https://developer.spotify.com/documentation/web-api/reference/playlists/get-playlists-tracks/

        Parameters:
            - user - the id of the user
            - playlist_id - the id of the playlist
            - fields - which fields to return
    """

    # first run through also retrieves total no of songs in library
    response = sp.user_playlist_tracks(user, playlist_id, fields=None, limit=100)
    results = response["items"]

    # subsequently runs until it hits the user-defined limit or has read all songs in the library
    while len(results) < response["total"]:
        response = sp.user_playlist_tracks(
            user, playlist_id, fields=fields, limit=100, offset=len(results)
        )
        results.extend(response["items"])
    
    return results

def get_tracks_full(songs):
    results = []
    while len(results) < len(songs):
        response = (sp.tracks(songs[len(results):len(results) + 50]))['tracks']
        response = list(filter(None, response))
        results.extend(response)
    return results

def get_audio_features_full(songs):
    results = []
    while len(results) < len(songs):
        response = sp.audio_features(songs[len(results):len(results) + 100])
        response = list(filter(None, response))
        results.extend(response)
    return results

<h1> Time to process each user in a huge for-loop </h1> <h3> (this takes a while) </h3>

In [58]:
all_averages = []

for selected_user in stripped_usernames[1:11]:
    playlists = []
    LIMIT = 20

    query = sp.user_playlists(selected_user, limit=LIMIT)

    n = 0

    while len(query['items']) != 0:
        n += LIMIT
        for i in query['items']:
            playlists.append({'title': i['name'],'id': i['id'], 'length': i['tracks']['total']})
        if LIMIT == len(query['items']):
            query = sp.user_playlists(selected_user, limit=LIMIT, offset=n)
        else:
            break
            
    songs = []

    for p in playlists:

        results = sp.playlist(p['id'])

        extended_results = user_playlist_tracks_full(user = selected_user, playlist_id = results['id']) # more than 100

        for r in extended_results:
            try:
                songs.append(r['track']['id']) 
            except:
                pass
    
    songs_meta = get_tracks_full(songs)
    
    song_info = {'id': [], 'title': [], 'artist': [], 'album': [], 
                 'explicit': [], 'popularity': []}

    for i in songs_meta:
        # append id to id column
        song_info['id'].append(i['id'])

        # append song title to title column
        song_info['title'].append(i['name'])

        # append artists to artist column
        s = ', '
        artists = s.join([name['name'] for name \
                         in i['artists']])
        song_info['artist'].append(artists)

        # append album title to album column
        song_info['album'].append(i['album']['name'])

        # append explicit info to explicit column
        song_info['explicit'].append(i['explicit'])

        # append popularity info to popularity column
        song_info['popularity'].append(i['popularity'])

    # convert song_info dictionary -> Pandas DataFrame
    song_info_df = pd.DataFrame.from_dict(song_info)

    song_info_df = song_info_df.drop_duplicates(subset=['id'])
    song_info_df = song_info_df.dropna()
    
    features_df = pd.DataFrame.from_dict(get_audio_features_full(songs))
    features_df = features_df.drop_duplicates(subset=['id'])
    features_df = features_df.dropna()
    
    df = song_info_df.merge(features_df)
    
    features = ['popularity','danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']

    feature_averages = []

    for feature in features:
        avg =  df[feature].mean()
        feature_averages.append(avg)
   
    all_averages.append(feature_averages)


<h1> These are the two datasets we want you to practice visualizations with! </h1>

sorry the labels do not include the questions—check the top of this notebook to match them up!

In [60]:
user_stats_df = pd.DataFrame(all_averages,
                  columns = ['popularity','danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo'])

user_stats_df

,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,62.205128,0.692436,0.809462,4.974359,-5.094949,0.641026,0.182456,0.071144,0.001701,0.256131,0.606026,132.294077
1,60.890173,0.684918,0.608226,5.078035,-7.085861,0.658960,0.133515,0.240063,0.027794,0.186895,0.450523,125.001205
2,50.659489,0.535571,0.653748,5.290713,-7.516564,0.806191,0.059469,0.244422,0.100112,0.202700,0.532035,123.427318
3,28.840708,0.443567,0.408822,5.197640,-13.863027,0.560472,0.080902,0.551557,0.383943,0.175904,0.340200,108.685959
4,45.867341,0.605338,0.566542,5.243902,-8.596186,0.624033,0.081538,0.429531,0.230805,0.166541,0.451694,121.980213
5,39.610515,0.605247,0.631628,5.274142,-7.577883,0.555794,0.082638,0.293000,0.153779,0.180435,0.450686,120.535787
6,51.597430,0.706270,0.616019,5.250535,-6.546577,0.561028,0.146713,0.207423,0.002014,0.164118,0.496746,119.392257
7,50.105033,0.542713,0.468726,5.002188,-10.749884,0.789934,0.046285,0.476762,0.073601,0.175743,0.541245,114.999527
8,64.847222,0.679556,0.604200,5.298611,-7.149861,0.500000,0.163600,0.240491,0.053422,0.217819,0.427974,119.839444
9,28.989676,0.491675,0.438530,5.023599,-12.198109,0.712389,0.042596,0.518497,0.173137,0.170102,0.406539,115.589034


In [61]:
first_ten_students_df = students_df[0:10]
first_ten_students_df 

,index,Progress,Finished,Q1,Q2,Q3,Q4,Q7,Q5,Q6,...,Q8_4,Q9,Q10,Q11_1,Q11_2,Q11_3,Q11_4,Q11_5,Q12,username
0,3,100,True,(AS) College of Arts and Sciences,Computer Science,"Undergraduate, 1st year (2026)",Male,Other club/team,International,Korea South,...,4,https://open.spotify.com/user/6cjzkudbvmwfe963...,https://open.spotify.com/playlist/5Kwx1SSB0pol...,NaN,https://open.spotify.com/playlist/37HUliOloWGE...,NaN,https://open.spotify.com/playlist/37HUliOloWGE...,https://open.spotify.com/playlist/5Kwx1SSB0pol...,NaN,6cjzkudbvmwfe963clvemzwkj
1,4,100,True,(AS) College of Arts and Sciences,Government,"Undergraduate, 1st year (2026)",Female,Professional club,International,Indonesia,...,5,https://open.spotify.com/user/vqlx3sn88fh9295c...,https://open.spotify.com/playlist/6huxxOvKkbRu...,NaN,NaN,NaN,NaN,NaN,NaN,vqlx3sn88fh9295cpn4z7toip
2,7,100,True,(ENG) College of Engineering,Cognitive Science,"Undergraduate, 1st year (2026)",Male,"Project team,Other club/team",Domestic,NaN,...,4,https://open.spotify.com/user/m3l6ktgr5im6yorm...,https://open.spotify.com/playlist/3Tp8L5xWN4OQ...,https://open.spotify.com/playlist/4ueOZx7jYJC1...,NaN,NaN,NaN,NaN,NaN,m3l6ktgr5im6yormbf36etw78
3,8,100,True,(ILR) School of Industrial and Labor Relations,Industrial and Labor Relations,"Undergraduate, 3rd year (2024)",Female,"Professional club,Other club/team",Domestic,NaN,...,7,https://open.spotify.com/user/t4llwr7d08bca91r...,https://open.spotify.com/playlist/76R0Tg5FCTrb...,NaN,NaN,NaN,NaN,NaN,NaN,t4llwr7d08bca91rqn1ro2aau
4,9,100,True,(ENG) College of Engineering,Computer Science,"Undergraduate, 4th year (2023)",Male,Project team,Domestic,NaN,...,3,https://open.spotify.com/user/ejlity?si=B7MbIe...,https://open.spotify.com/playlist/07epaq8mPbCB...,NaN,NaN,NaN,NaN,NaN,NaN,ejlity
5,13,100,True,(AS) College of Arts and Sciences,Mathematics,"Undergraduate, 2nd year (2025)",Female,"Professional club,Other club/team",Domestic,NaN,...,5,https://open.spotify.com/user/aaa9rnpre89bf2i0...,https://open.spotify.com/playlist/4WjCgwiirz98...,NaN,https://open.spotify.com/playlist/75P8aVPlabMc...,NaN,NaN,NaN,NaN,aaa9rnpre89bf2i0tx7ihyem0
6,15,100,True,(AS) College of Arts and Sciences,English,"Undergraduate, other",Female,"Professional club,Other club/team",Domestic,NaN,...,7,https://open.spotify.com/user/deadpan.typicaln...,https://open.spotify.com/playlist/6O8Gd241tsCl...,https://open.spotify.com/playlist/3cgTaDVasXgE...,https://open.spotify.com/playlist/3tGQUB0EjqFy...,NaN,https://open.spotify.com/playlist/6HfiVdghosyM...,https://open.spotify.com/playlist/3nUAsSk8rEWK...,NaN,deadpan.typicalnerdbird1492
7,16,100,True,(CALS) College of Agriculture and Life Sciences,Information Science,"Undergraduate, 4th year (2023)",Female,Other club/team,Domestic,NaN,...,2,https://open.spotify.com/user/fluttershyfatima...,https://open.spotify.com/playlist/48ufj7347Mnw...,NaN,NaN,NaN,NaN,NaN,NaN,fluttershyfatima
8,17,100,True,(AS) College of Arts and Sciences,Government,"Undergraduate, 3rd year (2024)",Female,Other club/team,Domestic,NaN,...,8,https://open.spotify.com/user/1283949946?si=nj...,https://open.spotify.com/playlist/3FLTO5QNCWTg...,https://open.spotify.com/playlist/6M9RziV7gkFz...,https://open.spotify.com/playlist/32hpqnOCX6YU...,https://open.spotify.com/playlist/4BIbSL8yR154...,https://open.spotify.com/playlist/2x2kvO5kP1e3...,https://open.spotify.com/playlist/0PwT4umg8ZSa...,NaN,1283949946
9,18,100,True,(ENG) College of Engineering,Operations Research and Engineering,"Undergraduate, 3rd year (2024)",Male,"Professional club,Other club/team",Domestic,NaN,...,3,https://open.spotify.com/user/31m3onadeh4wxucg...,https://open.spotify.com/playlist/26zMq20Djd7R...,https://open.spotify.com/playlist/7F0HcjeuLDVL...,NaN,NaN,NaN,NaN,NaN,31m3onadeh4wxucgckv3fhcyeoqu
